In [2]:
#pip install qdrant_client


In [1]:
import numpy as np

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.http import models

from qdrant_client import QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams, Distance

client = QdrantClient(host="sprakbankdb1.lx.nb.no", port=6333)


In [3]:
import sqlite3

In [9]:
db = "../Bildevektor/all_images.db"

In [10]:
collection_name = "images_1900_cos"

In [11]:



# Collection configuration
vector_dimension = 512  # Replace with your vectors' dimension
distance_metric = Distance.COSINE  # The distance metric for the vector space (COSINE, EUCLIDEAN, etc.)

# Attempt to create the collection with the correct vectors configuration
try:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=vector_dimension,
            distance=distance_metric,
        )
    )
    print(f"Collection '{collection_name}' successfully created.")
except Exception as e:
    print(f"An error occurred: {e}")


Collection 'images_1900_cos' successfully created.


In [12]:
def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        r = con.execute(sql, params).fetchall()
    return r

In [15]:
res = query(db, "select * from metadata limit 2")

In [25]:
res[0][1].split('/')[6]
res[0][1].split('/')[7]

'120,9,2298,3036'

In [26]:
%%time
def fetch_data_from_sqlite(db, batch_size=100):
    offset = 0
    while True:
        with sqlite3.connect(db) as con:
            sql = "SELECT rowid, * FROM words ORDER BY rowid LIMIT ? OFFSET ?"
            batch = con.execute(sql, (batch_size, offset)).fetchall()
        if not batch:
            break
        yield batch
        offset += len(batch)  # Ensure the offset increases by the actual number of rows fetched

def fetch_images_from_sqlite(db, batch_size=100):
    offset = 0
    while True:
        with sqlite3.connect(db) as con:
            sql = "SELECT * FROM metadata ORDER BY rowid LIMIT ? OFFSET ?"
            batch = con.execute(sql, (batch_size, offset)).fetchall()
        if not batch:
            break
        yield batch
        offset += len(batch)  # Ensure the offset increases by the actual number of rows fetched

def convert_vector(binary_vector):
    # Assuming binary_vector is a blob from SQLite, convert it to a list
    return np.frombuffer(binary_vector, dtype=np.float32).tolist()

for batch in fetch_images_from_sqlite(db=db, batch_size=100):
    # Use list comprehension to construct PointStruct objects for each record in the batch
    points = [
        PointStruct(
            id=record[0],  # Assuming record[0] is the rowid
            vector=convert_vector(record[2]),  # Assuming record[2] is the binary vector
            payload={"url": record[1], "urn":record[1].split('/')[6], "coord":record[1].split('/')[7]}  # Assuming record[1] is the word
        )
        for record in batch
    ]

    # Use upsert method to insert or update points in Qdrant
    client.upsert(
        collection_name=collection_name,
        points=points
    )

CPU times: user 2min 22s, sys: 11.7 s, total: 2min 34s
Wall time: 7min 10s
